# How It Works

In this notebook, we explain the algorithm behind `ethik` and how it is implemented.

Let's assume you have a dataset and a model that generates predictions on this dataset. For the sake of the example, we'll be wortking with the "Adult" dataset. This dataset contains a binary label indicating if a person's annual income is larger than $50k per year. The data is available on the [UCI machine learning repository](https://archive.ics.uci.edu/ml/index.php).

In [73]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from sklearn import model_selection

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
names = [
    'age', 'workclass', 'fnlwgt', 'education',
    'education-num', 'marital-status', 'occupation',
    'relationship', 'race', 'gender', 'capital-gain',
    'capital-loss', 'hours-per-week', 'native-country',
    'salary'
]
dtypes = {
    'workclass': 'category',
    'education': 'category',
    'marital-status': 'category',
    'occupation': 'category',
    'relationship': 'category',
    'race': 'category',
    'gender': 'category',
    'native-country': 'category'
}

X = pd.read_csv(url, names=names, header=None, dtype=dtypes)
X['gender'] = X['gender'].str.strip().astype('category')  # Remove leading whitespace
y = X.pop('salary').map({' <=50K': False, ' >50K': True})

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, shuffle=True, random_state=42)

X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


`ethik` is model-agnostic and works with the data only.

In [74]:
def get_black_box_model_predictions():
    import lightgbm as lgb
    model = lgb.LGBMClassifier(random_state=42).fit(X_train, y_train)
    return model.predict_proba(X_test)[:, 1]

y_pred = get_black_box_model_predictions()
# We use a named pandas series to make plot labels more explicit
y_pred = pd.Series(y_pred, name='>$50k')
y_pred

0       0.009978
1       0.366950
2       0.749978
3       0.015823
4       0.015583
          ...   
8136    0.004573
8137    0.113812
8138    0.001476
8139    0.707632
8140    0.261709
Name: >$50k, Length: 8141, dtype: float64

`ethik` lets us explore how each feature impact the model's behavior, either its average prediction or its performance.

## The Maths

Mathematically, we have a dataset of $n$ samples and $p$ features:

$$
\begin{aligned}
X &= \{(x_i^1, x_i^2, \ldots, x_i^p) \}_{1 \leq i \leq n} \\
&= \{x_i\}_{1 \leq i \leq n}
\end{aligned}
$$

Usually, $X$ is `X_test`. Whether to use the training or the test set is out of the scope of this notebook and is discussed [here](https://christophm.github.io/interpretable-ml-book/feature-importance.html).

We also have the model's predictions:

$$
\begin{aligned}
\hat{Y} &= \{f(x_i)\}_{1 \leq i \leq n} \\
&= \{y_i\}_{1 \leq i \leq n}
\end{aligned}
$$

We potentially also have $Y$, the true output observed from the real world.

The dataset $(X, \hat{Y}, Y)$ can be considered as an empirical probability distribution $Q_n$ (the unobserved true distribution being $Q$). As explained in the [paper](https://arxiv.org/pdf/1810.07924v2.pdf), the key idea of `ethik` is to stress this distribution with respect to a property of $X$. For instance, we can ask:

*How can we stress the distribution so that the mean of the feature $X^j$ is $\mu_t$ instead of $\mu$?*

Let's take an example. First, let's build a toy dataset with two features $X_{age}$ and $X_{hours-per-week}$:

In [75]:
n = len(X_test) 
ds = np.array([
    [X_test["age"].iloc[i], X_test["hours-per-week"].iloc[i], y_pred[i], int(y_test.iloc[i])]
    for i in range(n)
])
ds

array([[2.70000000e+01, 3.80000000e+01, 9.97846722e-03, 0.00000000e+00],
       [4.50000000e+01, 4.00000000e+01, 3.66949539e-01, 0.00000000e+00],
       [2.90000000e+01, 5.50000000e+01, 7.49978306e-01, 1.00000000e+00],
       ...,
       [2.50000000e+01, 4.00000000e+01, 1.47576107e-03, 0.00000000e+00],
       [5.00000000e+01, 4.00000000e+01, 7.07631637e-01, 1.00000000e+00],
       [2.40000000e+01, 4.00000000e+01, 2.61709249e-01, 1.00000000e+00]])

Let's plot the [marginal distribution](https://en.wikipedia.org/wiki/Marginal_distribution) of $X_{age}$:

In [76]:
def mean_from_pdf(x, density):
    return sum(x * density) / sum(density)

def plot_pdf(x, density):
    width = x[1:] - x[:-1]
    mean = mean_from_pdf(x, density)
    return go.Figure().add_bar(
        x=x,
        y=density,
        width=width,
    ).update_layout(
        plot_bgcolor="#FFF",
        xaxis=dict(
            title="age",
            linecolor="black",
        ),
        yaxis=dict(
            title="Density",
            linecolor="black",
        ),
        shapes=[
            go.layout.Shape(
                type="line",
                x0=mean,
                y0=0,
                x1=mean,
                y1=1,
                yref="paper",
                line=dict(
                    color="red",
                    width=1,
                    dash="dash",
                )
            )
        ],
        annotations=[
            go.layout.Annotation(
                x=mean,
                y=1,
                xref="x",
                yref="paper",
                text=f"Mean = {mean:.2f}",
                showarrow=False,
                yanchor="top",
                bgcolor="#FFF",
                bordercolor="red",
                borderpad=4,
            )
        ]
    )

In [77]:
ages = ds[:, 0]
densities, bin_edges = np.histogram(ages, bins=40, density=True)
x_deltas = bin_edges[1:] - bin_edges[:-1]
x_pdf = bin_edges[:-1] + x_deltas / 2
y_pdf = densities
plot_pdf(x_pdf, densities)

Like with [counterfactual explanations](https://christophm.github.io/interpretable-ml-book/counterfactual.html), we now ask the question:

*What if the mean age was $\mu_t$?*

For instance, we can stress $Q_n$ so that $\mu_t = 20$:

In [78]:
stressed_densities = np.copy(densities)
stressed_densities[0] = 0.68
stressed_densities[1] = 0.15
for i, x in enumerate(x_pdf):
    if x > 20:
        stressed_densities[i] /= 7
plot_pdf(x_pdf, stressed_densities)

We could sample from this distribution and observe how the model behaves for a mean age of 20. It'd be really similar to [Partial Dependence Plots](https://christophm.github.io/interpretable-ml-book/pdp.html). 

The problem here is that we only stressed the marginal distribution of $X_{age}$. But features may be correlated (if I have a PhD, I'm probably not 20) and if we don't consider that, we are likely to generated unrealistic individuals (a 18-year-old who has a PhD).

To address this issue, the key idea behind the paper is to **stress the whole distribution** $(X, \hat{Y}, Y)$ with respect to the mean of one feature **while minimizing the distance to the original distribution**. The [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) is used.

Contrary to what was done above, we stress the whole distribution $(X, \hat{Y}, Y)$, not just the marginal distribution of a feature $X_j$.

Minimizing the Kullback–Leibler divergence between the stressed distribution and the originally-observed distribution lets us believe that we are not creating individuals that are too unrealistic. Of course, reaching a minimum doesn't mean that this minimum is actually small and, to certify the model, we need to define a threshold on the distance. This is out of the scope of this notebook though.

As explained in the [paper](https://arxiv.org/pdf/1810.07924v2.pdf), we can efficiently compute a unique set of weights $\{\lambda_i^{(t)}\}_{1 \leq i \leq n}$ to stress the probability density function to reach a criterion while minimizing the KL divergence:

$$
Q_t = \frac{1}{n}\sum_{i=1}^n \lambda_i^{(t)} \delta_{X_i, \hat{Y}_i, Y_i}.
$$

with:

* $Q_t$ being the new distribution meeting the criterion $t$ (e.g. $E[X_{age}] = \mu_t$) and minimizing $KL(Q_t, Q_n)$ ;
* $\delta_{X_i, \hat{Y}_i, Y_i}$ being the probability density of the $i$-th sample.

The theorem that tells us how to compute these weights works for a wide class of deformations but the package only implements deformations on the mean of a single variable for now: $E[X_{j0}] = \mu_t$.

## The code

To learn how to use the package, please refer to the tutorials and the API reference. Here, we explain how it works internally.

`ethik` provides different classes that inherit from `ethik.explainer.Explainer`:

In [79]:
import ethik

explainer = ethik.ClassificationExplainer()

The explainer stores the computation results in its `.info` attribute, which is a pandas dataframe (firstly empty):

In [80]:
explainer.info

,feature,tau,value,lambda,label,bias,bias_low,bias_high


When we ask for an explanation, *i.e.* for an analysis of how some features impact the model's output, the `.info` is completed:

In [81]:
explainer.explain_bias(
    X_test=X_test["age"],
    y_pred=y_pred
)
explainer.info

,feature,tau,value,lambda,label,bias,bias_low,bias_high
0,age,-1.00,19.000000,-0.507662,>$50k,0.005798,0.005798,0.005798
1,age,-0.95,19.976612,-0.352627,>$50k,0.014250,0.014250,0.014250
2,age,-0.90,20.953224,-0.268180,>$50k,0.026014,0.026014,0.026014
3,age,-0.85,21.929837,-0.214814,>$50k,0.039703,0.039703,0.039703
4,age,-0.80,22.906449,-0.177631,>$50k,0.054331,0.054331,0.054331
5,age,-0.75,23.883061,-0.149886,>$50k,0.069283,0.069283,0.069283
6,age,-0.70,24.859673,-0.128117,>$50k,0.084192,0.084192,0.084192
7,age,-0.65,25.836285,-0.110375,>$50k,0.098834,0.098834,0.098834
8,age,-0.60,26.812898,-0.095480,>$50k,0.113075,0.113075,0.113075
9,age,-0.55,27.789510,-0.082677,>$50k,0.126831,0.126831,0.126831


Because `explainer.n_samples = 1` (the default), we don't compute the confidence interval and `bias = bias_low = bias_high`.

By default, caching is disabled and the `.info` is reset before each explanation:

In [82]:
# No "age" anymore here
# `.explain_*()` returns the filled `.info`
explainer.explain_bias(
    X_test=X_test[["education-num", "hours-per-week"]],
    y_pred=y_pred
)

,feature,tau,value,lambda,label,bias,bias_low,bias_high
0,education-num,-1.00,5.000000,-0.598658,>$50k,0.094360,0.094360,0.094360
1,education-num,-0.95,5.252709,-0.568326,>$50k,0.098050,0.098050,0.098050
2,education-num,-0.90,5.505417,-0.539281,>$50k,0.102000,0.102000,0.102000
3,education-num,-0.85,5.758126,-0.511214,>$50k,0.106208,0.106208,0.106208
4,education-num,-0.80,6.010834,-0.483862,>$50k,0.110678,0.110678,0.110678
...,...,...,...,...,...,...,...,...
77,hours-per-week,0.80,56.118020,0.064397,>$50k,0.303078,0.303078,0.303078
78,hours-per-week,0.85,57.088515,0.066726,>$50k,0.304374,0.304374,0.304374
79,hours-per-week,0.90,58.059010,0.068968,>$50k,0.305492,0.305492,0.305492
80,hours-per-week,0.95,59.029505,0.071134,>$50k,0.306446,0.306446,0.306446


The `.plot_*()` methods just call their corresponding `.explain_*()` method and plot the data by returning a `plotly.graph_objs.Figure()` object:

In [83]:
explainer.plot_bias(
    X_test=X_test["age"],
    y_pred=y_pred
)

In [84]:
explainer.info.head()

,feature,tau,value,lambda,label,bias,bias_low,bias_high
0,age,-1.00,19.000000,-0.507662,>$50k,0.005798,0.005798,0.005798
1,age,-0.95,19.976612,-0.352627,>$50k,0.014250,0.014250,0.014250
2,age,-0.90,20.953224,-0.268180,>$50k,0.026014,0.026014,0.026014
3,age,-0.85,21.929837,-0.214814,>$50k,0.039703,0.039703,0.039703
4,age,-0.80,22.906449,-0.177631,>$50k,0.054331,0.054331,0.054331


When caching is enabled, the explanations are stacked into the `.info`:

In [85]:
cache_explainer = ethik.ClassificationExplainer(memoize=True)
cache_explainer.explain_bias(
    X_test=X_test["age"],
    y_pred=y_pred
)

,feature,tau,value,lambda,label,bias,bias_low,bias_high
0,age,-1.00,19.000000,-0.507662,>$50k,0.005798,0.005798,0.005798
1,age,-0.95,19.976612,-0.352627,>$50k,0.014250,0.014250,0.014250
2,age,-0.90,20.953224,-0.268180,>$50k,0.026014,0.026014,0.026014
3,age,-0.85,21.929837,-0.214814,>$50k,0.039703,0.039703,0.039703
4,age,-0.80,22.906449,-0.177631,>$50k,0.054331,0.054331,0.054331
5,age,-0.75,23.883061,-0.149886,>$50k,0.069283,0.069283,0.069283
6,age,-0.70,24.859673,-0.128117,>$50k,0.084192,0.084192,0.084192
7,age,-0.65,25.836285,-0.110375,>$50k,0.098834,0.098834,0.098834
8,age,-0.60,26.812898,-0.095480,>$50k,0.113075,0.113075,0.113075
9,age,-0.55,27.789510,-0.082677,>$50k,0.126831,0.126831,0.126831


In [86]:
cache_explainer.explain_bias(
    X_test=X_test["education-num"],
    y_pred=y_pred
)

KeyError: 'age'

`.explain_*()` filters the `.info` to return the queried features only. But internally, the results are appended to the dataframe:

In [ ]:
cache_explainer.info

When we explain performance, columns are added to the `.info` to store the results:

In [ ]:
from sklearn import metrics

cache_explainer.explain_performance(
    X_test=X_test["age"],
    y_test=y_test,
    y_pred=y_pred > 0.5,
    metric=metrics.accuracy_score,
)

In [ ]:
cache_explainer.explain_performance(
    X_test=X_test["age"],
    y_test=y_test,
    y_pred=y_pred,
    metric=metrics.log_loss,
)